In [24]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [25]:
# Load the annotations
columns = ['A', 'B', 'C', 'D', 'E','F']
train_df = pd.read_csv('../data/train.csv',names = columns)
test_df = pd.read_csv('../data/test.csv',names = columns)
validation_df = pd.read_csv('../data/valid.csv',names = columns)


In [26]:
train_df.head()

,A,B,C,D,E,F
0,1c64bff45216007f_jpg.rf.00472dc1dcd49262a84274...,45,58,402,355,Truck
1,b61201eee8903342_jpg.rf.00edd486466508ba478ce7...,216,44,407,378,Bus
2,b61201eee8903342_jpg.rf.00edd486466508ba478ce7...,44,48,235,356,Bus
3,bdc6ddcd3b5129bb_jpg.rf.00da54335aeba68b1d59d9...,41,163,155,303,Bus
4,bdc6ddcd3b5129bb_jpg.rf.00da54335aeba68b1d59d9...,143,136,416,350,Bus


In [27]:
test_df.head()

,A,B,C,D,E,F
0,6e85d4ef31b5eed3_jpg.rf.01ea5ec813abb42ce95665...,0,65,366,385,Car
1,6e85d4ef31b5eed3_jpg.rf.01ea5ec813abb42ce95665...,362,148,397,205,Car
2,4ecafc9a0bae8003_jpg.rf.2UbsFgyK4QxCrSNix6wM.jpg,23,72,398,348,Truck
3,d66aabbb1f7299cc_jpg.rf.27774c8e4188c1c8b62075...,90,102,166,146,Car
4,d66aabbb1f7299cc_jpg.rf.27774c8e4188c1c8b62075...,116,118,188,146,Car


In [28]:
validation_df.head()

,A,B,C,D,E,F
0,ac03aa25bc294ae9_jpg.rf.yhDIMFwNrWa52ylx6Z2I.jpg,68,104,307,332,Ambulance
1,532953d0fc03d7ac_jpg.rf.zwev1qoVdbZlhhNFG7Do.jpg,0,10,49,50,Car
2,532953d0fc03d7ac_jpg.rf.zwev1qoVdbZlhhNFG7Do.jpg,30,40,384,348,Car
3,532953d0fc03d7ac_jpg.rf.zwev1qoVdbZlhhNFG7Do.jpg,216,12,277,47,Car
4,4a11e1503e4317a1_jpg.rf.xr2mlTpcmvzGODyFyQTf.jpg,36,0,416,415,Bus


In [29]:
# Base directory for your dataset
base_dir = '../data'

# Adjust these paths based on your directory structure
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Load CSV files
train_df = pd.read_csv(os.path.join(base_dir, 'train.csv'), names=['filename', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])
validation_df = pd.read_csv(os.path.join(base_dir, 'valid.csv'), names=['filename', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])
test_df = pd.read_csv(os.path.join(base_dir, 'test.csv'), names=['filename', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])

# Preprocessing function for Inception v3
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

# Data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Adjust batch size according to your system's capability
batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(416, 416),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_df,
    directory=validation_dir,
    x_col='filename',
    y_col='label',
    target_size=(416, 416),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col='label',
    target_size=(416, 416),
    batch_size=batch_size,
    class_mode='categorical')


Found 1676 validated image filenames belonging to 5 classes.
Found 454 validated image filenames belonging to 5 classes.
Found 258 validated image filenames belonging to 5 classes.


In [35]:
from kerastuner import HyperModel
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

class MyHyperModel(HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        # Load the InceptionV3 model
        base_model = InceptionV3(input_shape=(416, 416, 3), include_top=False, weights='imagenet', pooling='avg')
        base_model.trainable = False

        # Hyperparameters to tune
        units = hp.Int('units', min_value=32, max_value=512, step=32)
        dropout_rate = hp.Float('dropout_rate', min_value=0, max_value=0.5, step=0.1)
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')

        # Building the model
        x = base_model.output
        x = Dense(units, activation='relu')(x)
        x = Dropout(dropout_rate)(x)  # Add a dropout layer
        predictions = Dense(self.num_classes, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=predictions)

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model


/var/folders/4k/gdz1h9lx32zg806p1cylzsw00000gn/T/ipykernel_5069/1993301932.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [37]:
from kerastuner.tuners import Hyperband

hypermodel = MyHyperModel(num_classes)

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    directory='my_dir',  # Directory to save the search logs
    project_name='vehicle_classification')

# Early stopping to prevent overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Start the search
tuner.search(train_generator,
             steps_per_epoch=steps_per_epoch,
             epochs=10,
             validation_data=validation_generator,
             validation_steps=validation_steps,
             callbacks=[stop_early])


Trial 30 Complete [00h 28m 33s]
val_accuracy: 0.8303571343421936

Best val_accuracy So Far: 0.8526785969734192
Total elapsed time: 06h 05m 43s


In [38]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
best_model.evaluate(test_generator)


9/9 [==============================] - 24s 3s/step - loss: 0.3392 - accuracy: 0.8837


[0.3392336964607239, 0.8837209343910217]

In [40]:
best_model.save('model1.h5')

/opt/anaconda3/envs/vehicle/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [41]:
best_model.save('my_model1.keras')

In [44]:
# Get the best trial
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

# Get the best hyperparameters
best_hyperparameters = best_trial.hyperparameters.values
print("Best Hyperparameters:", best_hyperparameters)


Best Hyperparameters: {'units': 128, 'dropout_rate': 0.1, 'learning_rate': 0.0023327793733607717, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
